In [ ]:
from utils.basic import loadBlatOutput
refFlat_path="/Users/dowonkim/Dropbox/data/UCSC/hg38/refFlat/refFlat_200817.txt"
blat = loadBlatOutput(refFlat_path, by='transID')

In [ ]:

def _get_transInfo(transid):
    return [t for t in blat[transid]
            if len(t['chrom'].split('_')) == 1][0]

def get_data(transid):
    transInfo = _get_transInfo(transid=transid)
    anti = '+' if transInfo['strand'] == '-' else '-'
    chrom = transInfo['chrom']
    start = transInfo['txnSta']
    end   = transInfo['txnEnd']
    return transInfo

In [6]:
transid="NM_002415"
get_data(transid=transid)

{'transName': 'MIF',
 'transID': 'NM_002415',
 'chrom': 'chr22',
 'chrNum': '22',
 'strand': '+',
 'txnSta': 23894382,
 'txnEnd': 23895223,
 'txnLen': 841,
 'cdsSta': 23894474,
 'cdsEnd': 23895106,
 'exnList': [(23894382, 23894582), (23894771, 23894944), (23895039, 23895223)],
 'exnLenList': [200, 173, 184],
 'exnLen': 557,
 'cdsList': [(23894474, 23894582), (23894771, 23894944), (23895039, 23895106)],
 'utr5': [(23894382, 23894474)],
 'utr3': [(23895106, 23895223)],
 'utr5Len': 92,
 'utr3Len': 117,
 'cdsLen': 348,
 'intron': [(23894582, 23894771), (23894944, 23895039)]}

In [7]:
# ------------------------------------------------------------------
# Drop-in replacement for getSpanningRegion ― identical output
# ------------------------------------------------------------------

def spanning_regions(exons, chrom, strand, k=20):
    """
    Parameters
    ----------
    exons : list[tuple[int, int]]
        0-based, right-open exon intervals, sorted & non-overlapping.
    chrom : str
        Chromosome label, e.g. 'chr1'.
    strand : str
        '+' or '-'.
    k : int, default 20
        Window length (bp).

    Returns
    -------
    list[tuple[str, ...]]
        Each element is a k-tuple of locus strings
        "chr:start-end<strand>" that together span ≥2 exons.
    """
    # 1. convert to 1-based, right-closed
    exons_1 = [(s + 1, e) for s, e in exons]
    lens    = [e - s + 1 for s, e in exons_1]

    result = []
    for i in range(len(exons_1) - 1):
        covered = 0
        for j in range(i + 1, len(exons_1)):
            covered += lens[j]
            if covered > k - 1:                # first j making sum ≥ k
                pair = exons_1[i : j + 1]      # exons i … j  (inclusive)
                result.extend(_emit_windows(pair, chrom, strand, k))
                break
    return result


# ------------------------------------------------------------------
# Helper functions (internal use only)
# ------------------------------------------------------------------
def _emit_windows(pair, chrom, strand, k):
    """Generate all k-bp windows inside an exon pair."""
    first_len_m1 = pair[0][1] - pair[0][0]  # (len − 1)

    if first_len_m1 > k - 1:                # long first exon
        pos, steps = pair[0][1] - (k - 2), 2 * (k - 1)
    else:                                   # short/medium first exon
        pos, steps = pair[0][0], 2 * first_len_m1

    tokens = []
    try:
        for _ in range(steps):              # collect genome coords
            tokens.append(pos)
            pos = _next_bp(pos, pair)
    except IndexError:
        pass                                # reached end of pair

    out = []
    for i in range(len(tokens) - k + 1):    # sliding window, 1-bp stride
        out.append(_tokens_to_locus(tokens[i : i + k], chrom, strand))
    return out


def _next_bp(pos, pair):
    """Advance 1 bp within pair, jumping introns if needed."""
    for idx, (s, e) in enumerate(pair):
        if s <= pos < e:                    # inside exon
            return pos + 1
        if pos == e:                        # exon boundary
            if idx == len(pair) - 1:
                raise IndexError            # past last exon
            return pair[idx + 1][0]         # next exon start
    raise IndexError                        # pos not in pair


def _tokens_to_locus(toks, chrom, strand):
    """Convert contiguous coord list to tuple of locus strings."""
    segs, start = [], toks[0]
    for prev, curr in zip(toks, toks[1:]):
        if curr != prev + 1:                # gap ⇒ new segment
            segs.append(f"{chrom}:{start}-{prev}{strand}")
            start = curr
    segs.append(f"{chrom}:{start}-{toks[-1]}{strand}")
    return tuple(segs)


# ------------------------------------------------------------------
# (Optional) quick parity check with original getSpanningRegion
# ------------------------------------------------------------------
if __name__ == "__main__":
    blat_data = get_data(transid=transid)
    exons, chr_, strand_, k_ = blat_data['exnList'], blat_data['chrom'], blat_data['strand'], 20
    region = spanning_regions(exons, chr_, strand_, k_)
       

In [7]:
print(exons)
region

[(23894382, 23894582), (23894771, 23894944), (23895039, 23895223)]


[('chr22:23894564-23894582+', 'chr22:23894772-23894772+'),
 ('chr22:23894565-23894582+', 'chr22:23894772-23894773+'),
 ('chr22:23894566-23894582+', 'chr22:23894772-23894774+'),
 ('chr22:23894567-23894582+', 'chr22:23894772-23894775+'),
 ('chr22:23894568-23894582+', 'chr22:23894772-23894776+'),
 ('chr22:23894569-23894582+', 'chr22:23894772-23894777+'),
 ('chr22:23894570-23894582+', 'chr22:23894772-23894778+'),
 ('chr22:23894571-23894582+', 'chr22:23894772-23894779+'),
 ('chr22:23894572-23894582+', 'chr22:23894772-23894780+'),
 ('chr22:23894573-23894582+', 'chr22:23894772-23894781+'),
 ('chr22:23894574-23894582+', 'chr22:23894772-23894782+'),
 ('chr22:23894575-23894582+', 'chr22:23894772-23894783+'),
 ('chr22:23894576-23894582+', 'chr22:23894772-23894784+'),
 ('chr22:23894577-23894582+', 'chr22:23894772-23894785+'),
 ('chr22:23894578-23894582+', 'chr22:23894772-23894786+'),
 ('chr22:23894579-23894582+', 'chr22:23894772-23894787+'),
 ('chr22:23894580-23894582+', 'chr22:23894772-23894788+'

[(23894382, 23894582), (23894771, 23894944), (23895039, 23895223)]

In [1]:
#from utils.basic import loadBlatOutput
from utils.rna import RNAcofold2, containCommonSNP, containGquad2, countCpG, GCcontent
from utils.basic import loadBlatOutput

from jklib.genome import locus
from jklib.bioDB import CommonSNP


refFlat_path="/Users/dowonkim/Dropbox/data/UCSC/hg38/refFlat/refFlat_200817.txt"
blat = loadBlatOutput(refFlat_path, by='transID')
def _get_transInfo(transid):
    return [t for t in blat[transid]
            if len(t['chrom'].split('_')) == 1][0]

def get_data(transid):
    transInfo = _get_transInfo(transid=transid)
    anti = '+' if transInfo['strand'] == '-' else '-'
    chrom = transInfo['chrom']
    start = transInfo['txnSta']
    end   = transInfo['txnEnd']
    return transInfo


def getlocInfo(loc):
    sequence = loc.twoBitFrag().upper()
    locStr   = loc.toString()
    loc_tmp = locus(f'{loc.chrom}:{loc.chrSta+1}-{loc.chrEnd-1}{loc.strand}')
    flagL = [ e[3] for e in loc_tmp.regionType()]
    flag = []
    for a in flagL:
        #print(a[0][0])
        if (('cds' in a[0][0]) or ('utr' in a[0][0])): #exon flag ['cds','utr']
            flag.append('e') 
        elif 'int' in a[0][0] : #intron
            flag.append('i')

    regionT = '/'.join(map(str, loc.regionType()))
    flag = ':'.join(flag)
    #hx_score = hx.hexamer_sum(locus(f'{loc.chrom}:{loc.chrSta-5}-{loc.chrEnd+5}{self.transInfo["strand"]}').twoBitFrag('hg38').upper())
    #hx_score = [hx_score[st] for st in ['5p_exon','5p_intron','3p_exon','3p_intron']]
    #hx_score = [None]*4
    homo, mono = RNAcofold2(sequence)
    return [flag,
            #transInfo['transName'],
            #transInfo['']
            locStr, sequence, len(sequence), regionT,
            containCommonSNP(loc),
            containGquad2(sequence),
            countCpG(sequence),
            GCcontent(sequence),
            homo, mono]

In [2]:
transid="NM_002415"

def _tile_region(transid, tile_length=17):
    
    transInfo = get_data(transid=transid)
    anti = '+' if transInfo['strand'] == '-' else '-'
    chrom = transInfo['chrom']
    start = transInfo['txnSta']
    end   = transInfo['txnEnd']
    L     = tile_length
    print(start, end)
    return [locus(f"{chrom}:{i}-{i+L-1}{anti}")   # inclusive end = i+L-1
        for i in range(start, end-L+1)]
    
tile_loc_list = _tile_region(transid=transid, tile_length=17)

locinfoL = [getlocInfo(tile_loc) for tile_loc in tile_loc_list ] 

23894382 23895223
(-3.954258918762207, -0.37198126316070557)
(-3.9293038845062256, -0.2338075488805771)
(-4.000546932220459, -0.3112214207649231)
(-4.142286777496338, -0.36899036169052124)
(-5.905167102813721, -1.7853741645812988)
(-6.736757278442383, -2.1914939880371094)
(-6.787779808044434, -2.202791213989258)
(-6.787387847900391, -2.202022075653076)
(-6.967465877532959, -2.2067830562591553)
(-5.010594844818115, -1.1828478574752808)
(-3.400317430496216, -0.2927590012550354)
(-4.7090559005737305, -0.2188580483198166)
(-4.9976325035095215, -0.21737252175807953)
(-7.53887939453125, -0.8690648078918457)
(-8.317544937133789, -1.023807406425476)
(-8.67871379852295, -1.0623793601989746)
(-8.722541809082031, -1.0463660955429077)
(-8.73337173461914, -1.0466039180755615)
(-9.837247848510742, -1.1834776401519775)
(-10.347404479980469, -1.338124394416809)
(-10.459209442138672, -1.4006106853485107)
(-10.706961631774902, -1.5803704261779785)
(-11.900238037109375, -1.1833115816116333)
(-11.95839500

In [4]:
locinfoL[0], locinfoL[-1]

(['i:e',
  'chr22:23894382-23894398-',
  'CTTCTCGGACACCACTG',
  17,
  "('GSTTP2', 'NR_003082', 'sense', [('lnc_intron4', (89724, 104181))])/('MIF', 'NM_002415', 'antisense', [('utr5', (-1, 76))])/('MIF-AS1', 'NR_038911', 'sense', [('lnc_exon3', (672, 1134))])",
  False,
  False,
  1,
  0.5882352941176471,
  -3.954258918762207,
  -0.37198126316070557],
 ['i:e',
  'chr22:23895206-23895222-',
  'AGTCTCTAAACCGTTTA',
  17,
  "('GSTTP2', 'NR_003082', 'sense', [('lnc_intron4', (90548, 103357))])/('MIF', 'NM_002415', 'antisense', [('utr3', (99, 1))])/('MIF-AS1', 'NR_038911', 'sense', [('lnc_exon3', (1496, 310))])",
  False,
  False,
  1,
  0.3529411764705882,
  -5.268829345703125,
  -0.042043138295412064])

In [ ]:
flagL = [ e[3] for e in tile_loc_list[0].regionType()]
flag = []
for a in flagL:
    print(a[0][0])
    if 'cu' in a[0][0] : #exon flag ['cds','utr']
        flag.append('e') 
    elif 'int' in a[0][0] : #intron
        flag.append('i')

In [14]:
test_seq

'TAGTCTCTAAACCGTTTATTTCTCCCCACCAGAAGGTTGGGGTGGGCGGGCCTAGAACACAGCGTGCGGCGGGTTCCCGGGTGGAGCCAGCGCAGACAGCGTGGGTCCCTGCGGCTCTTAGGCGAAGGTGGAGTTGTTCCAGCCCACATTGGCCGCGTTCATGTCGTAATAGTTGATGTAGACCCTGCGGGGGGAGGAGGCCGGACTCAGCGGGTGGCTCAGTCCCGGGCCTGGCCGCGCGCCGCCGCCCCTCCCCCGCCCCTCCGCGACTCCGCGTACCTGTCCGGGCTGATGCGCAGGCGCTCGGCCAGCAGGCCGCACAGCAGCTTGCTGTAGGAGCGGTTCTGCGCGCCGCCGATCTTGCCGATGCTGTGCAGGCTGCAGAGCGCGCACGGCTCGCTGGAGCCGCCGAAGGCCATGAGCTGGTCCGGGACCACGTGCACCGCGATGTACTGCGAGGAAAGGGCGCGGTCAGCCCGCCCCAGCGACCTCGTCGGGCCCCGAACGTCCACTTCGGGCCCGAGCCACCGTCCTCCCCCCGCCCCGCCCCCCAGCTCCGTTCAGGAGTCGCCTCCCCAGCTCCCAGCGCGGAACCCCTCGTCCGGTGGGCACCCCCCTCTTCCTGTCCCCTCCCGGCAAACCTGGGGGGGCTTGCCGGTGGCCTGCGCCAGCTGCTGGGTGAGCTCGGAGAGGAACCCGTCCGGCACGGAGGCGCGGGGCACGTTGGTGTTTACGATGAACATCGGCATGATGGCAGAAGGACCAGGAGACCCGCGCAGAGGCACAGACGCACGCGCCGCGGCCGCCGCTGAGCTACGTGCCTGACTTCTCGGACACCACTG'

In [22]:
test_seq = locus("chr22:23894382-23895222-").twoBitFrag(assembly='hg38')
start = 23894382
end=23895223
L=17
print(end-start-L+1, end-start)
test_array = []
for i in range(0, end-start-L+1):
    test_array.append(test_seq[i:i+L])

test_array[::-1]
    


825 841


['CTTCTCGGACACCACTG',
 'ACTTCTCGGACACCACT',
 'GACTTCTCGGACACCAC',
 'TGACTTCTCGGACACCA',
 'CTGACTTCTCGGACACC',
 'CCTGACTTCTCGGACAC',
 'GCCTGACTTCTCGGACA',
 'TGCCTGACTTCTCGGAC',
 'GTGCCTGACTTCTCGGA',
 'CGTGCCTGACTTCTCGG',
 'ACGTGCCTGACTTCTCG',
 'TACGTGCCTGACTTCTC',
 'CTACGTGCCTGACTTCT',
 'GCTACGTGCCTGACTTC',
 'AGCTACGTGCCTGACTT',
 'GAGCTACGTGCCTGACT',
 'TGAGCTACGTGCCTGAC',
 'CTGAGCTACGTGCCTGA',
 'GCTGAGCTACGTGCCTG',
 'CGCTGAGCTACGTGCCT',
 'CCGCTGAGCTACGTGCC',
 'GCCGCTGAGCTACGTGC',
 'CGCCGCTGAGCTACGTG',
 'CCGCCGCTGAGCTACGT',
 'GCCGCCGCTGAGCTACG',
 'GGCCGCCGCTGAGCTAC',
 'CGGCCGCCGCTGAGCTA',
 'GCGGCCGCCGCTGAGCT',
 'CGCGGCCGCCGCTGAGC',
 'CCGCGGCCGCCGCTGAG',
 'GCCGCGGCCGCCGCTGA',
 'CGCCGCGGCCGCCGCTG',
 'GCGCCGCGGCCGCCGCT',
 'CGCGCCGCGGCCGCCGC',
 'ACGCGCCGCGGCCGCCG',
 'CACGCGCCGCGGCCGCC',
 'GCACGCGCCGCGGCCGC',
 'CGCACGCGCCGCGGCCG',
 'ACGCACGCGCCGCGGCC',
 'GACGCACGCGCCGCGGC',
 'AGACGCACGCGCCGCGG',
 'CAGACGCACGCGCCGCG',
 'ACAGACGCACGCGCCGC',
 'CACAGACGCACGCGCCG',
 'GCACAGACGCACGCGCC',
 'GGCACAGA

In [13]:
locus("chr22:23894382-23894398-").twoBitFrag()

'CTTCTCGGACACCACTG'

In [20]:

len("AGTCTCTAAACCGTTTA")

17